<a href="https://colab.research.google.com/github/AlpacaJake/OpenProject/blob/master/InfLearn_Lab_9_1_XOR_tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 9-1 XOR and Tensorboard usage


In [0]:
LOG_DIR = 'drive/data/tb_logs'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
	
import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
	  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))
	
get_ipython().system_raw('./ngrok http 6006 &')
	
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-12-03 08:06:22--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.174.75.106, 54.165.216.26, 52.201.75.180, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.174.75.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.1MB/s    in 0.9s    

2019-12-03 08:06:24 (14.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://9c9e8ef2.ngrok.io


In [0]:
import tensorflow as tf
import numpy as np

In [0]:
tf.set_random_seed(777)



In [0]:
x_data = np.array([[0,0], [0,1],[1,0],[1,1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype = np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])


In [0]:
with tf.name_scope("layer0") as scope:
  W0 = tf.Variable(tf.random_normal([2,10]), name = 'weight0')
  b0 = tf.Variable(tf.random_normal([10]), name = 'bias0')
  layer0 = tf.sigmoid(tf.matmul(X, W0) + b0)

  W0_hist = tf.summary.histogram("weights0", W0)
  b0_hist = tf.summary.histogram("biases0", b0)
  layer0_hist = tf.summary.histogram("layer0", layer0)
   

with tf.name_scope("layer1") as scope:

  W1 = tf.Variable(tf.random_normal([10,10]), name = 'weight1')
  b1 = tf.Variable(tf.random_normal([10]), name = 'bias1')
  layer1 = tf.sigmoid(tf.matmul(layer0, W1) + b1)

  W1_hist = tf.summary.histogram("weights1", W1)
  b1_hist = tf.summary.histogram("biases1", b1)
  layer1_hist = tf.summary.histogram("layer1", layer1)

with tf.name_scope("layer2") as scope:
  W = tf.Variable(tf.random_normal([10, 1]), name = 'weight')
  b = tf.Variable(tf.random_normal([1]), name = 'bias')
  hypothesis = tf.sigmoid(tf.matmul(layer1, W) + b)

  W_hist = tf.summary.histogram("weights2", W)
  b_hist = tf.summary.histogram("biases2", b)
  hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

# sigmoid corss entropy function
with tf.name_scope("Cost") as scope:
  cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y) * tf.log(1-hypothesis))
  cost_hist = tf.summary.scalar("cost", cost)

with tf.name_scope("Train") as scope:
  train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)



<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  merged_summary = tf.summary.merge_all()
  writer = tf.summary.FileWriter(LOG_DIR)
  writer.add_graph(sess.graph)
   
  for step in range(10001):
    _, summary, cost_val, w_val = sess.run([train, merged_summary, cost, W], feed_dict={X: x_data, Y: y_data})
        
    writer.add_summary(summary, global_step = step)
    if step % 100 ==0:
      print(step, cost_val, w_val)

  h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict = {X: x_data, Y: y_data})
  print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccruacy: ", a)
  

0 0.7230154 [[ 0.20832591]
 [-0.9339557 ]
 [-1.2575641 ]
 [ 1.5100675 ]
 [ 0.34970373]
 [ 1.1178005 ]
 [-0.46789223]
 [ 0.73509413]
 [ 0.26310262]
 [-0.6919164 ]]
100 0.68436253 [[ 0.1943553 ]
 [-1.0020254 ]
 [-1.2659713 ]
 [ 1.464976  ]
 [ 0.2966958 ]
 [ 1.06071   ]
 [-0.54703736]
 [ 0.6718013 ]
 [ 0.21278211]
 [-0.6936175 ]]
200 0.6796745 [[ 0.23311715]
 [-1.0176618 ]
 [-1.274642  ]
 [ 1.4934572 ]
 [ 0.31557134]
 [ 1.0583174 ]
 [-0.57511735]
 [ 0.67832935]
 [ 0.1814699 ]
 [-0.69178945]]
300 0.67397696 [[ 0.27803826]
 [-1.0339    ]
 [-1.2942643 ]
 [ 1.5295782 ]
 [ 0.3431184 ]
 [ 1.0601755 ]
 [-0.6116374 ]
 [ 0.68536603]
 [ 0.15087588]
 [-0.69367164]]
400 0.66669947 [[ 0.33022326]
 [-1.0503147 ]
 [-1.3274093 ]
 [ 1.5751339 ]
 [ 0.38027933]
 [ 1.0670953 ]
 [-0.6577608 ]
 [ 0.6930806 ]
 [ 0.12086444]
 [-0.6992365 ]]
500 0.6571001 [[ 0.39090556]
 [-1.0664781 ]
 [-1.3775612 ]
 [ 1.6324846 ]
 [ 0.42845368]
 [ 1.0800592 ]
 [-0.71502745]
 [ 0.7015173 ]
 [ 0.09128167]
 [-0.70849246]]
600 0.644

In [0]:
%ls

drive/  ngrok*  ngrok-stable-linux-amd64.zip  sample_data/


In [0]:
%cd drive/data/tb_logs

/content/drive/data/tb_logs


In [0]:
%ls

events.out.tfevents.1575360408.a1dee0cc97ac
